In [4]:
from tqdm.notebook import tqdm
import datetime
import dpkt
import numpy as np
import matplotlib.pyplot as plt
import os
import open3d as op3 
import pandas as pd
import time
from scipy.sparse.csgraph import connected_components
from open3d import JVisualizer
from BfTableGenerator import RansacCollector
from matplotlib import cm
from sklearn.cluster._dbscan_inner import dbscan_inner
from numpy.lib.stride_tricks import sliding_window_view
from DDBSCAN import Raster_DBSCAN
from scipy import ndimage 
import cv2
from scipy.optimize import linear_sum_assignment

theta_raw = np.array([[-25,1.4],[-1,-4.2],[-1.667,1.4],[-15.639,-1.4],
                            [-11.31,1.4],[0,-1.4],[-0.667,4.2],[-8.843,-1.4],
                            [-7.254,1.4],[0.333,-4.2],[-0.333,1.4],[-6.148,-1.4],
                            [-5.333,4.2],[1.333,-1.4],[0.667,4.2],[-4,-1.4],
                            [-4.667,1.4],[1.667,-4.2],[1,1.4],[-3.667,-4.2],
                            [-3.333,4.2],[3.333,-1.4],[2.333,1.4],[-2.667,-1.4],
                            [-3,1.4],[7,-1.4],[4.667,1.4],[-2.333,-4.2],
                            [-2,4.2],[15,-1.4],[10.333,1.4],[-1.333,-1.4]
                            ])[:,0]
theta = np.sort(theta_raw)
azimuths = np.arange(0,360,0.2)

In [398]:
np.set_printoptions(suppress=True)

In [5]:
os.chdir(r'/Users/czhui960/Documents/Lidar/RawLidarData/FrameSamplingTest')
frame_set = np.arange(0,5000,1).astype('int')
collector = RansacCollector(pcap_path=r'./2020-7-27-10-30-0.pcap',frames_set = frame_set)
collector.gen_tdmap()
collector.gen_thredmap(d = 1.2,thred_s = 0.3,N = 20,delta_thred = 1e-3,step = 0.1,inuse_frame = frame_set)
# d = 1.2,thred_s = 0.3,N = 20,delta_thred = 1e-3,step = 0.1 FrameSampling


  0%|          | 2/5000 [00:00<04:39, 17.90it/s]

Loading pcap...


  0%|          | 0/32 [00:00<?, ?it/s]

Generating Threshold Map


100%|██████████| 32/32 [00:16<00:00,  1.97it/s]


In [6]:
Thred_map = collector.thred_map
db = Raster_DBSCAN(window_size=(5,13),eps = 1.5, min_samples= 16,Td_map_szie=Thred_map.shape)

In [7]:
Td_maps = []
Labeling_maps = []
Time_consumings = []
for i in tqdm(range(2000)):
    ti = time.time()
    Td_map = collector.td_maps[i].copy()
    Foreground_map = (Td_map < Thred_map)&(Td_map != 0)
    Labeling_map = db.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
    Td_maps.append(Td_map)
    Labeling_maps.append(Labeling_map)
    Time_consumings.append((time.time() - ti)*1000)

# Kalman Filter 

In [1043]:
def is_roll_over(mask):
    """
    Check if the mask has roll over problem 
    """
    
    flag = False
    for row in mask:
        if row[-1] & row[0]:
            flag = True
            break 
    return flag

In [1044]:
def extract_xywh(Labeling_map):
    unique_id = np.unique(Labeling_map)[1:]
    xywh = []
    for label in unique_id:
        mask = Labeling_map == label
        if is_roll_over(mask): # process roll over 
            temp = np.zeros_like(mask)
            temp[:,:900] = mask[:,900:]
            temp[:,900:] = mask[:,:900]
            x,y,w,h = cv2.boundingRect(temp.astype(np.uint8))
            x += 900
            xywh.append((x,y,w,h))
        else:
            xywh.append(cv2.boundingRect(mask.astype(np.uint8)))
    # convert to center point
    xywh = np.array(xywh)
    # convert left-up point to center point 
    # constrain all x values in 0-1800 
    xywh[:,0] = xywh[:,0] + xywh[:,2]/2
    xywh[:,1] = xywh[:,1] + xywh[:,3]/2
    xywh[:,0][xywh[:,0] > 1800] -= 1800
    xywh[:,0][xywh[:,0] < 0] += 1800
    
    return xywh,unique_id

In [1045]:
def extract_mea_state_vec(xywh):
    return xywh.reshape((-1,xywh.shape[1],1))

In [1046]:
def state_predict(A,Q,state,P):
    """
    state: s_k-1, (n x 8 x 1)
    Cov: P_k-1 (n x 8 x 8)
    """
    state_ = np.matmul(A,state)
    state_[:,0][state_[:,0] > 1800] -= 1800
    state_[:,0][state_[:,0] < 0] += 1800
    P_ = np.matmul(np.matmul(A,P),A.transpose()) + Q
    return state_,P_

def state_update(A,H,state_,P_,R,mea):
    """
    mea: m_k (m x 4 x 1)
    
    """
    K = np.matmul(np.matmul(P_,H.transpose()),np.linalg.inv(np.matmul(np.matmul(H,P_),H.transpose()) + R))
    P = np.matmul((np.eye(A.shape[0]) - np.matmul(K,H)),P_)
    residual = mea - np.matmul(H,state_) # n x 4 x 1
    state = state_ + np.matmul(K,residual)
    
    return state, P 

In [1047]:
def get_affinity_mat(state_,P_,mea):
    State_affinity = np.zeros((state_.shape[0],mea.shape[0]))
    for i,s_ in enumerate(state_):
        v = s_[:4].copy()
        VI = P_[i][:4,:4].copy()
        for j,m in enumerate(mea):
            u = m.copy()
            if np.abs(u[0][0] - v[0][0]) > 900:
                if u[0][0] > v[0][0]:
                    u[0] -= 900
                else:
                    v[0] -= 900
                    
            State_affinity[i][j] = distance.mahalanobis(u.flatten(),v.flatten(),VI)
    return State_affinity

In [1048]:
class detected_obj():
    def __init__(self):
        self.glb_id = None
        self.start_frame = None
        self.missing_count = 0 # frame count of out of detection
        self.P = None
        self.state = None 
        self.label_seq = [] # represented labels at each frame 
        

In [1052]:
A = np.array( # x,y,w,h,x',y',w',h'
    [[1,0,0,0,1,0,0,0],
     [0,1,0,0,0,1,0,0],
     [0,0,1,0,0,0,1,0],
     [0,0,0,1,0,0,0,1],
     [0,0,0,0,1,0,0,0],
     [0,0,0,0,0,1,0,0],
     [0,0,0,0,0,0,1,0],
     [0,0,0,0,0,0,0,1]]
)
Q = np.diag([10,10,1,1,1,1,1,1]) 
H = np.array([[1,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0],
              [0,0,0,1,0,0,0,0]])
R = np.diag([100,100,5,5])

In [1053]:
Off_tracking_pool = tracking(Q,A,H,R)

In [1027]:
Qs = [Q*i for i in range(1,10)] 
Rs = [R*i for i in [10,50,100,200,400]]

In [1028]:
Results = []
for i in tqdm(range(len(Qs))):
    for j in tqdm(range(len(Rs))):
        Results.append(tracking(Qs[i],A,H,R[j]))

[[  24.67792536  559.9187441  1849.51642328 1756.34051368  698.31296709]
 [ 589.96101566    9.16515139 2399.09107789 2306.75681423 1228.93368413]
 [1820.69602076 2385.15911419    6.4807407    88.26664149 1168.67360713]
 [1736.68736392 2302.00890528   90.14987521    6.4807407  1086.51507123]
 [ 669.38553913 1215.2468885  1177.8111054  1086.40876285    4.58257569]]
[[  14.02966573  372.77317858 1221.41253748 1154.88727839  456.94228018]
 [ 385.64155852    6.87988778 1592.93269339 1526.79171711  819.36104378]
 [1217.76699165 1591.4526884     6.0559911    72.04084491  780.12141396]
 [1151.99765671 1525.98257731   60.78185354    6.814683    715.14454063]
 [ 443.93451106  810.75964866  777.62687587  711.98829429    5.24534333]]
[[ 301.48187938    3.36224988  986.02808749  924.41824426  363.24978213]
 [   2.5338637   300.86298659 1287.56536769 1226.10321288  662.93888538]
 [1282.54572474  983.80650044    4.41391713   58.0257227   621.68930708]
 [1224.87365677  926.04936526   61.74499964    1.

[[ 608.6627347   112.11086836    0.74045461  529.75921497  624.92868371]
 [1232.539688    725.85616522  624.06156885   95.6134288     0.98469064]
 [1138.27890814  631.03599811  530.16958286    0.52490003   96.12660692]
 [   2.94276895  510.86654008  610.40080623 1139.59241874 1235.13335179]
 [ 500.92825148    8.65298317  118.48033769  638.59438163  734.65123332]]
[[ 135.78912628  629.02668533    0.82523503  528.58867609  625.95627418]
 [ 753.54122562 1253.94318386  625.74989473   97.78095978    1.60494165]
 [ 656.88241149 1157.84075374  529.99782669    1.8353772    98.22789269]
 [ 504.03796528    2.85074056  629.51306111 1158.37509044 1256.06577422]
 [   3.91750785  500.53614866  136.51590218  658.18018913  756.47154867]]
[[ 159.91966537  646.34358748    0.61125022  529.24323665]
 [ 780.69272509 1273.49162339  627.73348878   99.69554223]
 [ 681.02203935 1174.53829527  529.11736835    2.53216385]
 [ 497.3490934     2.83654914  647.49863603 1176.84060991]
 [   4.82372775  495.11320552  1

[[    6.35900034   592.27712651]
 [  586.34031399     3.96693008]
 [ 6077.93005133 12893.18433944]]
[[    3.57718463   620.01260352]
 [  618.51415856     3.22535719]
 [ 8365.84008311 16971.07577586]]
[[    7.09613648   652.7309694 ]
 [  646.77062031     4.20315389]
 [11426.82948724 21712.51515653]]
[[    7.38107469   678.28576579]
 [  670.21959485     1.92732433]
 [18621.79865523 24922.09364998]]
[[    8.61255933   701.52998214]
 [  694.6027288      0.44073908]
 [19605.61301627 26328.71604452]]
[[  2.09421145 719.31825005]
 [719.23522426   2.5870354 ]]
[[  2.95811231 742.66750678 778.23209837]
 [740.30525389   1.77494031  37.66330626]]
[[  3.48386868 765.0319412  799.47637009]
 [760.92748123   2.1278178   38.33855161]
 [632.03955572  30.           1.73205081]]
[[  5.24399755 785.03731692 823.64392492]
 [780.04662712   2.33461868  40.81583174]
 [755.88496087  34.49745025   4.1455018 ]]
[[  7.11004224 803.26256574 842.94604715]
 [796.71075338   2.44434976  42.61999584]
 [829.09495609  39

[[1532.1822429     3.76161633]
 [   3.48958641 1537.20409336]
 [ 847.65554673 6748.34825915]]
[[1566.91712297    2.29444651]
 [   2.07346875 1569.10620273]
 [ 984.76916836 8723.33205612]]
[[ 1596.03124968     2.7290957 ]
 [    8.96952114  1603.61066186]
 [ 1092.32862579 10861.44384059]]
[[ 1899.1289954   1623.5987072      2.41803043   750.91498581]
 [  268.84264331     8.72629764  1630.34164904   882.29499325]
 [  880.71651651  1178.62573364 13150.70881264  7538.1401455 ]]
[[1646.77374654    4.46905463  770.73188533]
 [   7.23192549 1655.77839935  884.76007828]
 [ 301.83482881   72.52728342  818.98426105]
 [ 707.00141443  643.29309028    1.73205081]]
[[   2.96885801 1677.18036357  792.75388995]
 [1680.94341059    1.63646376  887.22929503]
 [ 171.20355819 3075.32296149 1414.79347833]
 [ 749.30782875  821.84561946    4.21637021]]
[[   1.05847361  818.91616235]
 [1707.32393883  890.14477937]
 [ 326.14529228 2079.39462404]
 [ 813.50942371    3.79785713]]
[[   3.07998477  841.42081842]
 [29

[[   8.05158454  763.41727643 1289.29069284 1226.34118525]
 [1280.39060661  524.80720033    3.88525861   63.25595548]
 [1276.20246116 5379.53886667 8237.8840045  7882.79358942]
 [ 757.68007466    3.06146337  523.96107544  461.27477802]
 [ 686.09109996   76.88924514  604.26274286  541.31802689]
 [1217.32106654  462.028686     65.2975497     2.11624868]]
[[    7.92332764   781.23877588  1309.71596597  1235.78217228]
 [ 1301.27457296   527.6464609      3.856842      75.19118733]
 [ 1457.27634846  6535.79625252 10005.48940996  9495.52853729]
 [  777.94849335     4.87559515   524.25579602   450.66423924]
 [ 1200.21973241   125.49398268  1020.99422855   893.20214172]
 [ 1230.93781269   457.54685732    72.07805273     6.68072111]]
[[    8.71334311   802.27110003  1325.15326897   735.70800963
   1245.76796831]
 [ 1320.97916856   524.97898138     3.99234747   591.77638998
     83.81550182]
 [ 1635.28796999  7810.3225211  11877.96167956  7301.90225848
  11225.90847486]
 [  792.99693336     4.835

...

[[ 3163.8833832   3018.59141184  3140.81548116  3152.11909214
   3249.24467253  3110.38040966  3856.97260093  3180.20784189
   3152.25106894  3122.41418854]
 [  283.67134622  1565.75991406   160.11215515   324.81629509
   1085.33267511  1958.16428681  1044.41770148    45.2818097
   1871.54320646  1751.99103265]
 [ 1941.3292982    638.11496092   132.95240372    66.48509584
   1115.3749997    244.39930899  1156.23655921   238.18861291
   1608.00082604  1488.24085967]
 [10555.13241477 17632.27329704 20435.27117158 21250.35819756
  15112.9329706  19821.11368419 14890.09844774  9429.57213883
  19411.15822146 18749.40255086]
 [ 1576.92461365   286.767444     248.9907906    415.53888165
    746.92733099   137.98087628   787.56945583  1792.4496016
     41.71025487    80.54952633]
 [  511.21458643  1805.38249419   386.41959357   552.41015441
   1325.53014927   273.39528566  1284.68350462   280.46756218
    180.8385739     67.64584821]
 [  852.77421263   459.05128443   964.04708257  1125.6284839

[[ 436.01695713    2.2925673   102.2228913   985.78743749  123.94386408
  1026.25314983  492.38298796  241.68282349 1059.93769214  342.23822088
    29.27507424  324.47951115]
 [ 770.07041525  339.28008579  442.22736473  648.40873662  216.64205469
   688.7295563   154.44708732   99.03196302  722.07771805    2.39331348
   332.11243949   18.8537989 ]
 [1458.74040016 1025.51937553 1128.4296879    40.66100064  902.59311794
     0.03619825  534.60711418  787.55205252   34.2553965   686.87294765
  1020.27404709  706.08818891]
 [ 924.95233793  492.26388548  595.28707967  493.46243354  369.24264686
   533.84055634    0.7965437   253.72571002  567.31273646  153.37561828
   486.44948944  172.99455161]
 [ 443.40357649   36.75308214  120.41805566  975.24480352  116.55614896
  1015.61293404  481.04732276  228.16006764 1049.05971604  329.03416633
    10.4478947   310.29410771]
 [ 556.06915704  120.98722573  223.91476888  864.20890499    2.63957495
   904.65713141  370.6701171   121.13064172  938.3083

[[ 249.04354885    3.18409027  983.90912783   59.62880406 1028.12061829
   496.02897046   56.1499673   247.12749946  295.49811549]
 [1310.92513396 1059.91386625   79.74779211 1005.92988157   36.04311727
   567.74401041 1088.0746147   817.24915049  771.62138074]
 [2023.65552772  970.11258841 3391.80520419  801.05105987 3579.31271595
  1250.70712789 1049.3927592   279.37073204  358.65852263]
 [ 225.5413346    59.22236726 1010.83746118  108.22046692 1054.64857416
   521.59528273    3.72908232  272.56608834  317.80043331]
 [ 306.79845028   57.04903476  927.86886829    0.55425683  972.15953505
   441.20831225  104.87374677  194.23388225  243.65524297]
 [   1.22699146  251.17255677 1231.99262336  306.90583465 1276.11437196
   743.33794553  225.87567514  493.72109859  540.70284137]
 [ 741.80111992  491.33305577  490.86735898  438.499429    534.79981766
     4.75018042  518.92109192  247.97042829  202.75881433]
 [ 540.94968083  292.47887774  693.07910504  242.7943518   736.85074837
   203.8675

[[  857.98161222   485.41770356   715.52712579   360.9356423
    415.91845834   136.03867982   361.06602445   448.80693726]
 [ 1204.75068872   826.08570505  1059.0815619      6.62945809
     60.69506093   470.00401882   701.592094      96.54847074]
 [33421.76505205 31148.2051118  32641.81480807 27620.9260908
  27553.29836243 28910.8378122  30299.5923116  27377.11391756]
 [ 2440.04439197  5417.68302784  6137.84003565  3233.01454257
   3113.40323919  4455.09212836  5071.61437055  3077.04549205]
 [  147.64310384   235.23196595     3.8551975   1065.77885747
   1121.71721997   593.07441655   360.46270165  1156.6284996 ]
 [13399.91080472 13831.944568   13331.23824621 17809.47882502
  18142.10634799 15363.95342957 14301.34226645 18398.68018917]
 [    5.68735532   386.33772765   154.26306964  1219.5274736
   1275.69319214   743.44087609   510.34717027  1310.48468128]
 [  506.13693138   129.68602196   361.19073372   703.15378413
    759.05127044   229.69312884     4.41120329   793.7040061 ]
 [ 

KeyboardInterrupt: 

In [1032]:
def tracking(Q,A,H,R):  
    # Initialization 

    """ Temp for Tracking """
    Frame_ind = 0
    Tracking_pool = {}
    Off_tracking_pool = {}
    Global_id = 0 
    missing_thred = 10
#     """ Temp for Kalman Filter """
#     A = np.array( # x,y,w,h,x',y',w',h'
#         [[1,0,0,0,1,0,0,0],
#          [0,1,0,0,0,1,0,0],
#          [0,0,1,0,0,0,1,0],
#          [0,0,0,1,0,0,0,1],
#          [0,0,0,0,1,0,0,0],
#          [0,0,0,0,0,1,0,0],
#          [0,0,0,0,0,0,1,0],
#          [0,0,0,0,0,0,0,1]]
#     )
#     Q = np.diag([10,1,10,1,1,1,1,1]) 
#     H = np.array([[1,0,0,0,0,0,0,0],
#                   [0,1,0,0,0,0,0,0],
#                   [0,0,1,0,0,0,0,0],
#                   [0,0,0,1,0,0,0,0]])
#     R = np.diag([5,5,5,5])

    xywh_init,unique_label_init = extract_xywh(Labeling_maps[Frame_ind])
    Frame_ind += 1 
    mea_init = extract_mea_state_vec(xywh_init)
    # m: n x 8 x 1
    state_init = np.concatenate([mea_init,np.zeros_like(mea_init)],axis = 1)
    P_init = np.full((xywh_init.shape[0],8,8),np.eye(8)*10)

    for label,i in enumerate(unique_label_init):
        new_detection = detected_obj()
        new_detection.glb_id = Global_id
        new_detection.P = P_init[i]
        new_detection.state = state_init[i]
        new_detection.label_seq.append(label)
        new_detection.start_frame = 0
        Tracking_pool[Global_id] = new_detection
        Global_id += 1
    """ Loop Start"""


    state_cur,P_cur,glb_id_cur = state_init,P_init,unique_id_init 

    # MOT start, given the prediction from last frame, set up cost func to associate IDs between current and next frame 
    while Frame_ind < 2000:
        """
        Extract Matrix P and State of each tracklet in Current Tracking Pool
        """
        glb_ids,P_cur,state_cur = [],[],[]
        for glb_id in Tracking_pool.keys():
            glb_ids.append(glb_id)
            P_cur.append(Tracking_pool[glb_id].P)
            state_cur.append(Tracking_pool[glb_id].state)
        P_cur,state_cur = np.array(P_cur),np.array(state_cur)

        state_cur_,P_cur_ = state_predict(A,Q,state_cur,P_cur) # predict next state 

        xywh_next,unique_lebel_next = extract_xywh(Labeling_maps[Frame_ind]) # read observation at next frame 
        mea_next = extract_mea_state_vec(xywh_next)  
        State_affinity = get_affinity_mat(state_cur_,P_cur_,mea_next)
        associated_ind_glb,associated_ind_label = linear_sum_assignment(State_affinity)

        state,P = state_update(A,H,state_cur_[associated_ind_glb],P_cur_[associated_ind_glb],R,mea_next[associated_ind_label])

        for i,glb_ind in enumerate(associated_ind_glb):

            Tracking_pool[glb_ids[glb_ind]].state = state[i]
            Tracking_pool[glb_ids[glb_ind]].P = P[i]
            Tracking_pool[glb_ids[glb_ind]].label_seq.append(unique_lebel_next[associated_ind_label[i]])
            Tracking_pool[glb_ids[glb_ind]].missing_count = 0

        """
        Failed tracking and new detections
        """
        # in a but not in b
        failed_tracked_ind = np.setdiff1d(np.arange(len(glb_ids)),associated_ind_glb) 
        if len(failed_tracked_ind) > 0:
            for fid in failed_tracked_ind:

                if Tracking_pool[glb_ids[fid]].missing_count > missing_thred:
                    Off_tracking_pool[glb_ids[fid]] = Tracking_pool.pop(glb_ids[fid])
                else:
                    Tracking_pool[glb_ids[fid]].missing_count += 1
                    Tracking_pool[glb_ids[fid]].state = state_cur_[fid]
                    Tracking_pool[glb_ids[fid]].P = P_cur_[fid]
                    Tracking_pool[glb_ids[fid]].label_seq.append(-1)

        new_detection_ind = np.setdiff1d(np.arange(len(unique_lebel_next)),associated_ind_label)
        if len(new_detection_ind) > 0:
            for n_id in new_detection_ind:
                new_detection = detected_obj()
                new_detection.glb_id = Global_id
                new_detection.P = np.eye(8)
                new_detection.state = np.concatenate([mea_next[n_id], np.zeros_like(mea_next[n_id])])
                new_detection.label_seq.append(unique_lebel_next[n_id])
                new_detection.start_frame = Frame_ind
                Tracking_pool[Global_id] = new_detection
                Global_id += 1

        Frame_ind += 1 
    """
    Release all tracking obj into off tracking pool
    """
    release_ids = [glb_id for glb_id in Tracking_pool.keys()]
    for r_id in release_ids:
        Off_tracking_pool[r_id] = Tracking_pool.pop(r_id)
    return Off_tracking_pool

In [1055]:
np.random.seed(150)
color_map = np.random.random((100,3))
# color_map = np.concatenate([color_map,np.array([[1,1,1]])])

for f in range(2000):
    Td_map = Td_maps[f]
    Labeling_map = Labeling_maps[f].copy()
    td_freq_map = Td_map
    Xs = []
    Ys = []
    Zs = []
    Labels = []
    for i in range(td_freq_map.shape[0]):
        longitudes = theta[i]*np.pi / 180
        latitudes = azimuths * np.pi / 180 
        hypotenuses = td_freq_map[i] * np.cos(longitudes)
        X = hypotenuses * np.sin(latitudes)
        Y = hypotenuses * np.cos(latitudes)
        Z = td_freq_map[i] * np.sin(longitudes)
        Valid_ind =  (td_freq_map[i] != 0)&(td_freq_map[i]<collector.thred_map[i]) # None zero index
        Xs.append(X[Valid_ind])
        Ys.append(Y[Valid_ind])
        Zs.append(Z[Valid_ind])
        Labels.append(Labeling_map[i][Valid_ind])
    Xs = np.concatenate(Xs)
    Ys = np.concatenate(Ys)
    Zs = np.concatenate(Zs)
    Labels = np.concatenate(Labels).astype('int')
    Colors = np.zeros((len(Labels),3))
    for key in Off_tracking_pool:
        n = f - Off_tracking_pool[key].start_frame
        if (n >= 0) & (n < len(Off_tracking_pool[key].label_seq)):
            label_cur_frame = Off_tracking_pool[key].label_seq[n]
            if label_cur_frame != -1:
                Colors[Labels == label_cur_frame] = color_map[key%len(color_map)]
    pcd = op3.geometry.PointCloud()
    XYZ = np.concatenate([Xs.reshape(-1,1),Ys.reshape(-1,1),Zs.reshape(-1,1)],axis = 1)
    pcd.points = op3.utility.Vector3dVector(XYZ)
    pcd.colors = op3.utility.Vector3dVector(Colors)
    op3.io.write_point_cloud(r'./Output File/PcdSequence'+"/{}.pcd".format(f), pcd)
        
            

In [937]:
def visualize(Td_map,Labeling_map):
    np.random.seed(250)
    color_map = np.random.random((100,3))
    color_map = np.concatenate([color_map,np.array([[1,1,1]])])
    td_freq_map = Td_map
    Xs = []
    Ys = []
    Zs = []
    Labels = []
    for i in range(td_freq_map.shape[0]):
        longitudes = theta[i]*np.pi / 180
        latitudes = azimuths * np.pi / 180 
        hypotenuses = td_freq_map[i] * np.cos(longitudes)
        X = hypotenuses * np.sin(latitudes)
        Y = hypotenuses * np.cos(latitudes)
        Z = td_freq_map[i] * np.sin(longitudes)
        Valid_ind =  (td_freq_map[i] != 0)&(td_freq_map[i]<collector.thred_map[i]) # None zero index
        Xs.append(X[Valid_ind])
        Ys.append(Y[Valid_ind])
        Zs.append(Z[Valid_ind])
        Labels.append(Labeling_map[i][Valid_ind])


    Xs = np.concatenate(Xs)
    Ys = np.concatenate(Ys)
    Zs = np.concatenate(Zs)
    Labels = np.concatenate(Labels).astype('int')
    Colors = []
    for i in range(len(Labels)):
        Colors.append(color_map[Labels[i]])
    Colors = np.array(Colors)
    pcd = op3.geometry.PointCloud()
    XYZ = np.concatenate([Xs.reshape(-1,1),Ys.reshape(-1,1),Zs.reshape(-1,1)],axis = 1)
    pcd.points = op3.utility.Vector3dVector(XYZ)
    pcd.colors = op3.utility.Vector3dVector(Colors)
    visualizer = JVisualizer()
    visualizer.add_geometry(pcd)
    visualizer.show()

In [962]:
visualize(Td_maps[1900],Labeling_maps[1900])

JVisualizer with 1 geometries